In [ ]:
#######################################################################
##  Made by: Dr. Keungoui Kim
##  Title: Scitech data import from Dimensions
##  goal : 
##  Data set:  
##  Time Span:
##  Variables
##      Input: 
##      Output: 
##  Time-stamp: #  "Sun Jan 26 01:47:34 2020":  edited by awe kim ; code
##  Notice :
#######################################################################

In [1]:
### Library
import pandas as pd
import numpy as np
import math

In [117]:
### Directory Setting for Export
dir = "E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/" # Home

In [2]:
### Data Import
import dimcli
dimcli.login(key="792DDFAFCCA7478D8F37159F274A2783",
             endpoint="https://app.dimensions.ai/api/dsl/v2")

dsl = dimcli.Dsl()

Dimcli - Dimensions API Client (v0.9.9.1)
Connected to: <https://app.dimensions.ai/api/dsl/v2> - DSL v2.5
Method: manual login


In [156]:
### Extract patent data
pat_data = dsl.query_iterative("""search patents
                                  where publications is not empty and year = 2022                    
                                  return patents[id+year+priority_year+cpc+publications+publication_ids]""") #  limit 50000

Starting iteration with limit=1000 skip=0 ...
0-1000 / 12332 (10.04s)
1000-2000 / 12332 (5.32s)
2000-3000 / 12332 (4.10s)
3000-4000 / 12332 (4.69s)
4000-5000 / 12332 (4.85s)
5000-6000 / 12332 (6.19s)
6000-7000 / 12332 (4.84s)
7000-8000 / 12332 (4.53s)
8000-9000 / 12332 (4.95s)
9000-10000 / 12332 (5.99s)
10000-11000 / 12332 (8.05s)
11000-12000 / 12332 (3.11s)
12000-12332 / 12332 (2.19s)
===
Records extracted: 12332


In [161]:
pat_data_df = pat_data.as_dataframe()
print(pat_data_df.shape)
print(pat_data_df.id.nunique())

(12332, 6)
12332


In [162]:
pat_data_df.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_"+"2022"+".csv", index=False)
pat_data_df.head()

,id,cpc,priority_year,publication_ids,publications,year
0,WO-2022236335-A1,"[A61K38/177, A61P29/00, C07K14/70578, A61P37/0...",2021.0,"[pub.1027342753, pub.1009940936]","[{'doi': '10.1182/blood-2003-06-2031', 'id': '...",2022
1,WO-2022236333-A2,"[A61B3/0016, A61B3/0033]",2021.0,[pub.1065228315],"[{'doi': '10.1364/ol.35.000739', 'id': 'pub.10...",2022
2,WO-2022236331-A1,"[B05B12/18, B05B12/16, B05B14/00, B05B7/0408]",2021.0,[pub.1103595211],"[{'doi': '10.1002/adem.201701084', 'id': 'pub....",2022
3,WO-2022236308-A1,"[A61B5/0042, A61B34/30, G01R33/3806, G01R33/34...",2021.0,[pub.1092392383],"[{'doi': '10.1109/tmag.2017.2751001', 'id': 'p...",2022
4,WO-2022236297-A1,"[C11D3/48, C11D3/0068, C11D3/381, C11D7/40, C1...",2021.0,[pub.1080312358],"[{'doi': '10.1128/jb.96.2.479-486.1968', 'id':...",2022


In [25]:
### Import .csv files
# pat_data_df = pd.read_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_01.csv")

# from ast import literal_eval

# pat_data_df['publication_ids'] = pat_data_df['publication_ids'].apply(literal_eval) #convert to list type

In [163]:
pat_pub_df = pat_data_df[['id','publication_ids']]
pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)
pat_pub_df.head()

,id,publication_ids
0,WO-2022236335-A1,pub.1027342753
1,WO-2022236335-A1,pub.1009940936
2,WO-2022236333-A2,pub.1065228315
3,WO-2022236331-A1,pub.1103595211
4,WO-2022236308-A1,pub.1092392383


In [164]:
### Extract publication_ids
pub_list = pat_pub_df.publication_ids.unique()
pd.DataFrame(pub_list, columns=["publication_ids"]).to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pub_list_df_"+"2022"+".csv")

### Iterative Codes for extracting patent data
- Maximum 50,000 records
- dimcli query cannot recognize ''... so I had to write the loop for each jurisdiction --> fixed
- If I could convert dataframe's column into a big list with "" as elements, I would use Version 2.

In [ ]:
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'] 
offices = ['US','EP','WO']

for year in years:
    for office in offices:
    ### Patent set
    query = """search patents 
                where publications is not empty and jurisdiction = """+ '''"'''+office+'''"''' +""" and priority_year=""" + year + """ 
                return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query_iterative(query) 
    pat_df = pat_data.as_dataframe()

    # Export patent set
    pat_df.to_csv(dir+"pat_data_df_"+year+"_"+office+".csv", index=False)

    # Create publication set
    pat_pub_df = pat_df[['id','publication_ids']]
    pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)

    ## Export Publication List version 1. ALL
    pub_list = pat_pub_df.publication_ids.unique()
    pub_list = pd.DataFrame(pub_list, columns=["publication_ids"])
    pub_list.to_csv(dir+"pub_list_df_"+year+"_"+office+".csv", index=False)

    ## Export Publication List version 2. 50,000 each
    # pub_list = pat_pub_df.publication_ids.unique()
    # loop = math.ceil(pub_list.shape[0]/50000)+1
    # for i in range(1,loop):
    #     temp = pub_list.iloc[0:49999,]
    #     temp.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pub_list_df_"+year+"_US_"+str(i)+".csv")
    #     pub_list = pub_list.drop(pub_list.index[0:49999])

    ### Publication set
    # First record
    query = """search publications 
                where id = """+'''"'''+pub_list.publication_ids[0]+'''"'''+"""
                return publications[id+type+volume+year+issue+title+journal+authors]"""
    temp_data = dsl.query(query)    
    pub_df = temp_data.as_dataframe()

    for pub in range(1,len(pub_list.publication_ids)):
        query = """search publications 
                    where id = """+'''"'''+pub_list.publication_ids[pub]+'''"'''+"""
                    return publications[id+type+volume+year+issue+title+journal+authors+category_for]"""
        temp_data = dsl.query(query)    
        temp_data = temp_data.as_dataframe()
        
        # bind rows
        pub_df = pd.concat([pub_df, temp_data], axis=0)

        print(year, office, pub)

    # Save publication set
    pub_df.to_csv(dir+"pub_data_df_"+year+"_"+office+".csv")

In [11]:
### Check total counts

years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'] 
totalRecUS = []
totalRecEU = []
totalRecWO = []
for year in years:
    ### US
    query = """search patents 
                    where publications is not empty and jurisdiction in ["US"] and priority_year=""" + year + """ 
                    return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query(query) 
    totalRecUS.append(pat_data.count_total)

    ### EP
    query = """search patents 
            where publications is not empty and jurisdiction in ["EP"] and priority_year=""" + year + """ 
            return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query(query) 
    totalRecEUWO.append(pat_data.count_total)

    ### WO
    query = """search patents 
            where publications is not empty and jurisdiction in ["WO"] and priority_year=""" + year + """ 
            return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query(query) 
    totalRecEUWO.append(pat_data.count_total)

    print(year)

# Present result
pd.DataFrame({'year':years,'USRec':totalRecUS,'EUWORec':totalRecEUWO})

Returned Patents: 20 (total = 61554)
Time: 1.53s
Returned Patents: 20 (total = 42638)
Time: 1.44s
2010
Returned Patents: 20 (total = 64011)
Time: 6.10s
Returned Patents: 20 (total = 42616)
Time: 2.79s
2011
Returned Patents: 20 (total = 65138)
Time: 1.40s
Returned Patents: 20 (total = 40043)
Time: 6.10s
2012
Returned Patents: 20 (total = 67394)
Time: 1.40s
Returned Patents: 20 (total = 42498)
Time: 4.59s
2013
Returned Patents: 20 (total = 61421)
Time: 6.02s
Returned Patents: 20 (total = 39677)
Time: 1.43s
2014
Returned Patents: 20 (total = 57035)
Time: 4.63s
Returned Patents: 20 (total = 38526)
Time: 1.49s
2015
Returned Patents: 20 (total = 50138)
Time: 5.93s
Returned Patents: 20 (total = 40015)
Time: 6.11s
2016
Returned Patents: 20 (total = 43624)
Time: 5.92s
Returned Patents: 20 (total = 42727)
Time: 6.05s
2017
Returned Patents: 20 (total = 30421)
Time: 1.42s
Returned Patents: 20 (total = 46209)
Time: 4.63s
2018
Returned Patents: 20 (total = 19572)
Time: 1.50s
Returned Patents: 20 (to

,year,USRec,EUWORec
0,2010,61554,42638
1,2011,64011,42616
2,2012,65138,40043
3,2013,67394,42498
4,2014,61421,39677
5,2015,57035,38526
6,2016,50138,40015
7,2017,43624,42727
8,2018,30421,46209
9,2019,19572,42038


#### Alternative codes

In [ ]:
years = '2021'

for year in years:
    query = """search patents 
            where publications is not empty and year=""" + year + """
            return patents[id+year+priority_year+cpc+publications+publication_ids]"""
    pat_data = dsl.query_iterative(query) 
    pat_df = pat_data.as_dataframe()
    rangeIndex = int(temp.count_total/1000)+1

    for t in range(50,rangeIndex):
        query = """search patents 
            where publications is not empty and year=""" + year + """
            return patents[id+year+priority_year+cpc+publications+publication_ids+reference_ids+times_cited] limit 1000 skip """ + str(1000*t)
        temp_data = dsl.query(query)
        temp_df = temp_data.as_dataframe()
        # bind rows
        pat_df = pd.concat([pat_df, temp_df], axis=0)
        print(t)

    # Save patent set
    pat_data_df.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_"+year+".csv", index=False)

    # Save publication set
    pat_pub_df = pat_data_df[['id','publication_ids']]
    pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)

    # Extract publication_ids
    pub_list = pat_pub_df.publication_ids.unique()
    pd.DataFrame(pub_list, columns=["publication_ids"]).to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pub_list_df_"+year+".csv", index=False)

In [55]:
# %%dslloop my_data search publications for "malaria" return publications limit 500

UsageError: %%dslloop is a cell magic, but the cell body is empty. Did you mean the line magic %dslloop (single %)?


In [ ]:
##### EP #####
    query = """search patents 
                where publications is not empty and jurisdiction in ["EP"] and priority_year=""" + year + """ 
                return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query_iterative(query) 
    pat_df = pat_data.as_dataframe()

    # Save patent set
    pat_df.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_"+year+"_EP.csv", index=False)

    # Save publication set
    pat_pub_df = pat_df[['id','publication_ids']]
    pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)

    # Extract publication_ids
    pub_list = pat_pub_df.publication_ids.unique()
    pub_list = pd.DataFrame(pub_list, columns=["publication_ids"])

    loop = math.ceil(pub_list.shape[0]/50000)+1
    
    for i in range(1,loop):
        temp = pub_list.iloc[0:49999,]
        temp.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pub_list_df_"+year+"_EP_"+str(i)+".csv")
        pub_list = pub_list.drop(pub_list.index[0:49999])

    ##### WO #####
    query = """search patents 
                where publications is not empty and jurisdiction in ["WO"] and priority_year=""" + year + """ 
                return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query_iterative(query) 
    pat_df = pat_data.as_dataframe()

    # Save patent set
    pat_df.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_"+year+"_WO.csv", index=False)

    # Save publication set
    pat_pub_df = pat_df[['id','publication_ids']]
    pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)

    # Extract publication_ids
    pub_list = pat_pub_df.publication_ids.unique()
    pub_list = pd.DataFrame(pub_list, columns=["publication_ids"])

    loop = math.ceil(pub_list.shape[0]/50000)+1
    
    for i in range(1,loop):
        temp = pub_list.iloc[0:49999,]
        temp.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pub_list_df_"+year+"_WO_"+str(i)+".csv")
        pub_list = pub_list.drop(pub_list.index[0:49999])
